In [1]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

#paths
PPMI_zippath='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018.zip'
PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
def reading_csv_files(folder_path):
    # saving all the csv files in a dataframe
    import os
    import glob
    # glob.glob('*.csv') #find all the csv files in a pathname. 
    os.chdir(folder_path)
    csv_files = [i for i in glob.glob('*.csv')]

    # Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
    dict_files={}
    files_names=[]
    for files in csv_files:
    #for i, files in enumerate(csv_files): 
        df=pd.read_csv(files,engine='python')
        basepath, filename=ntpath.split(files)
        files_names.append(filename)
        dict_files[filename]=df

    # Calculating the # of rows and features for each dataframe and storing them in a list of tuples
    shape_df=[]
    for i in range(0,len(dict_files.keys())):
        nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
        shape_df.append((nrows,ncols))

    return (shape_df,files_names,dict_files)

## 1-Subject-Characteristics

In [3]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Screening__Demographics - useful

In [4]:
SCREEN=dict_files[files_names[3]]
#SCREEN.head()
#list(set(SCREEN['EVENT_ID']))
#SCREEN.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME',
       'CONSNTDT', 'APPRDX', 'CURRENT_APPRDX', 'P3GRP', 'BIRTHDT', 'GENDER',
       'HISPLAT', 'RAINDALS', 'RAASIAN', 'RABLACK', 'RAHAWOPI', 'RAWHITE',
       'RANOS', 'PRJENRDT']
SCREEN_sel=SCREEN[SEL]
SCREEN_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,CONSNTDT,APPRDX,CURRENT_APPRDX,P3GRP,BIRTHDT,GENDER,HISPLAT,RAINDALS,RAASIAN,RABLACK,RAHAWOPI,RAWHITE,RANOS,PRJENRDT
0,224392801,3400,CONSENT,SCREEN,06/2010,1.0,1.0,NaN,1971.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010
1,224394301,3401,CONSENT,SCREEN,06/2010,2.0,2.0,NaN,1954.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010
2,224398401,3402,CONSENT,SCREEN,06/2010,3.0,3.0,NaN,1964.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10/2011
3,224400201,3403,CONSENT,SCREEN,06/2010,1.0,1.0,NaN,1941.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010
4,224718101,3404,CONSENT,SCREEN,06/2010,2.0,2.0,NaN,1954.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010


## 2-Biospecimen

In [5]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/2-Biospecimen/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Skin_Biopsy - useful: Feat-bio (wound closure)

In [6]:
SKBIO=dict_files[files_names[1]]
#SKBIO.head()
#SKBIO.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT','SKBIOCMP','WOUNDCLS']
SKBIO_sel=SKBIO[SEL]
SKBIO_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,SKBIOCMP,WOUNDCLS
0,455646501,3429,V08,SKBIO,08/2014,1,1
1,455667101,3430,V08,SKBIO,09/2014,1,1
2,455864201,3444,V07,SKBIO,08/2014,1,1
3,456430101,3431,V08,SKBIO,09/2014,1,1
4,456815901,3406,V10,SKBIO,09/2014,1,2


### Genetic_Results_Tests - useful: Feat-bio

In [7]:
MUTRSLT=dict_files[files_names[3]]
#MUTRSLT.head()
#MUTRSLT.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'GENECAT', 'LRRKCD', 'MUTRSLT']
MUTRSLT_sel=MUTRSLT[SEL]
MUTRSLT_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,GENECAT,LRRKCD,MUTRSLT
0,426136501,41386,GMU,MUTRSLT,01/2014,1,1.0,1
1,445446201,50157,GMU,MUTRSLT,06/2014,1,1.0,1
2,422444501,40819,GMU,MUTRSLT,01/2014,1,1.0,1
3,423319501,40577,GMU,MUTRSLT,01/2014,1,1.0,1
4,493347201,41564,GMU,MUTRSLT,01/2015,1,1.0,1


### IPSC_Labs: useful - Feat-bio: blood, liver disease, heparin(?), myeloma(?) stuff (linked to Med dose history)

In [8]:
### ISPC_Labs: useful - Feat-bio
#### Select only history of liver disease and multiple myeloma (can be useful for treatments strategies)
IPSCLAB=dict_files[files_names[5]]
#IPSCLAB.head()
IPSCLAB.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'PDMEDDT', 'PDMEDTM', 'WARFBEF', 'HEPBEF', 'HXLIVER',
       'HXMMYLMA', 'BLDLTH', 'LTHTM', 'LTHINVN', 'BLDSST', 'BLDCPT']
# we might be able to remove some features. 
IPSCLAB_sel=IPSCLAB[SEL]
IPSCLAB_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PDMEDYN,PDMEDDT,PDMEDTM,WARFBEF,HEPBEF,HXLIVER,HXMMYLMA,BLDLTH,LTHTM,LTHINVN,BLDSST,BLDCPT
0,551588601,3409,U01,IPSCLAB,01/2016,1,01/2016,08:25:00,0,0,0,0,1,08:45:00,9.0,1,1
1,551594301,4051,U01,IPSCLAB,01/2016,1,01/2016,08:31:00,0,0,0,0,1,09:20:00,9.0,1,1
2,551939601,4105,U01,IPSCLAB,01/2016,0,NaN,NaN,0,0,0,0,1,11:50:00,9.0,1,1
3,554801901,3411,U01,IPSCLAB,02/2016,0,NaN,NaN,0,0,0,0,1,09:52:00,9.0,1,1
4,555320201,3419,U01,IPSCLAB,02/2016,0,NaN,NaN,0,0,0,0,1,12:10:00,9.0,1,1


### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
#### Understand if the cohort of patient IDs are different from the classic PPMI cohort.

In [9]:
### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
Biospecimen_Analysis=dict_files[files_names[6]]
#Biospecimen_Analysis.head()
#len(list(set(PILOTRSLT['PATNO'])))
#Biospecimen_Analysis.columns

In [81]:
SEL=['PATNO', 'DIAGNOSIS', 'CLINICAL_EVENT', 'TYPE', 'TESTNAME',
       'TESTVALUE','RUNDATE', 'PROJECTID']
Biospecimen_Analysis_sel=Biospecimen_Analysis[SEL]
Biospecimen_Analysis_sel.head()
#list(set(Biospecimen_Analysis_sel['TYPE']))

,PATNO,DIAGNOSIS,CLINICAL_EVENT,TYPE,TESTNAME,TESTVALUE,RUNDATE,PROJECTID
0,3400,PD,BL,Cerebrospinal Fluid,Abeta 42,207,2011-07-21,101
1,3400,PD,BL,Cerebrospinal Fluid,p-Tau181P,14,2011-07-21,101
2,3400,PD,BL,Cerebrospinal Fluid,Total tau,10,2011-07-21,101
3,3400,PD,ST,Cerebrospinal Fluid,Abeta 42,231,2011-07-15,101
4,3400,PD,ST,Cerebrospinal Fluid,p-Tau181P,21,2011-07-15,101


about Cerobrospinal fluid: https://en.wikipedia.org/wiki/Cerebrospinal_fluid

### Lumbar_Puncture_Sample_Collection - useful: Feat-bio

In [11]:
LUMBAR=dict_files[files_names[9]]
#LUMBAR.head()
#LUMBAR.columns

### Only features useful I think: white cell blood count, red blood cell count, total glucose, total protein.

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'FASTSTAT','PDMEDYN', 'PDMEDDT', 'PDMEDTM',
     'SMPDSCRD','RBCRSLT','WBCRSLT','TOPRRSLT','TGLCRSLT','SPFI','FLUORO']
LUMBAR_sel=LUMBAR[SEL]
LUMBAR_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,FASTSTAT,PDMEDYN,PDMEDDT,PDMEDTM,SMPDSCRD,RBCRSLT,WBCRSLT,TOPRRSLT,TGLCRSLT,SPFI,FLUORO
0,232556301,3401,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,0.0,1.0,20.0,63.0,0.0,0.0
1,232568401,3403,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,0.0,1.0,19.0,56.0,0.0,0.0
2,233649201,3400,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,76.0,1.0,75.0,55.0,0.0,0.0
3,235052301,3404,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,1.0,2.0,38.0,59.0,0.0,0.0
4,236619201,3405,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Blood_Chemistry__Hematology - Contains information about patient blood. 
### Ask doctors if really useful. 

In [82]:
COVANCE=dict_files[files_names[12]]
#COVANCE.head()

IndexError: list index out of range

### IUSM_Catalog - DNK - most likely useless
#### seems ti be only metadata on the lab collections.
Look @ the PPMI doc to understand what is this table

In [15]:
IUSMCAT=dict_files[files_names[14]]
#IUSMCAT.head()

## 3-Enrollment - useless?

In [92]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

In [94]:
### 'RBD_PSG_Eligibility.csv' - useless as just used for determining PPMI eligibility
ind_rbd_psg=dict_files[files_names[5]]
#ind_rbd_psg.head()
#list(set(ind_rbd_psg['protocol_id']))
#ind_rbd_psg.columns
## If useful: select the following columns:
#['PATNO', 'protocol_id','rbd_eligibility_results', 'date_eligibility_report',
       #'percent_rem_sleep', 'rem_sleep_without_atonia', 'rem_sleep_scored',
      # 'percent_withoutAtonia_epochs', 'percent_withoutAtonia_miniEpochs',
       #'psg_eval_activity']

### 'Primary_Diagnosis.csv' - useful: Feat-Gen

In [89]:
### 'Primary_Diagnosis.csv' - useful: Feat-Gen
PRIMDXPD=dict_files[files_names[8]]
#PRIMDXPD.head()
#PRIMDXPD.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PRIMDIAG', 'OTHNEURO']
PRIMDXPD_sel=PRIMDXPD[SEL]
PRIMDXPD_sel.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PRIMDIAG,OTHNEURO
0,226859501,V,3401,SC,PRIMDXPD,06/2010,17.0,NaN
1,232502501,V,3405,SC,PRIMDXPD,07/2010,17.0,NaN
2,226902401,V,3403,SC,PRIMDXPD,06/2010,1.0,NaN
3,232739901,V,3404,SC,PRIMDXPD,06/2010,17.0,NaN
4,232697801,V,3406,SC,PRIMDXPD,07/2010,1.0,NaN


### 5-Medical History

In [95]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Prodromal_Diagnostic_Questionnaire.csv': useful: Feat-Gen (PRIMDIAG & PSLVL2)
#### NB: PSLV2 evaluates the proba of the patient being PD: might be useful for early PD. 
#### But not to be used in the main analysis as a feature (only for Prodromal, and not measured at each timestamp). 

In [99]:
PRODIAG=dict_files[files_names[0]]
#PRODIAG.head()
#PRODIAG.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'PRIMDIAG',
       'OTHNEURO', 'PSLVL2']
PRODIAG_sel=PRODIAG[SEL]
#PRODIAG_sel.head()

### 'Diagnostic_Features.csv': useful: Feat-PD

In [21]:
DIAGFEAT=dict_files[files_names[1]]
DIAGFEAT.head()
#DIAGFEAT.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DFSTROKE', 'DFRSKFCT', 'DFPRESNT', 'DFRPROG', 'DFSTATIC', 'DFHEMPRK',
       'DFAGESX', 'DFOTHCRS', 'DFCRSCM', 'DFRTREMP', 'DFRTREMA', 'DFPATREM',
       'DFOTHTRM', 'DFTREMCM', 'DFRIGIDP', 'DFRIGIDA', 'DFAXRIG', 'DFUNIRIG',
       'DFTONE', 'DFOTHRIG', 'DFRIGCM', 'DFBRADYP', 'DFBRADYA', 'DFAKINES',
       'DFBRPLUS', 'DFOTHABR', 'DFABRCM', 'DFPGDIST', 'DFGAIT', 'DFFREEZ',
       'DFFALLS', 'DFOTHPG', 'DFPGCM', 'DFPSYCH', 'DFCOGNIT', 'DFDYSTON',
       'DFCHOREA', 'DFMYOCLO', 'DFOTHHYP', 'DFHYPCM', 'DFHEMTRO', 'DFPSHYPO',
       'DFSEXDYS', 'DFURDYS', 'DFBWLDYS', 'DFOCULO', 'DFEYELID', 'DFNEURAB',
       'DFDOPRSP', 'DFRAPSPE', 'DFBULBAR', 'DFCTSCAN', 'DFMRI', 'DFATYP']
DIAGFEAT_sel=DIAGFEAT[SEL]
DIAGFEAT_sel.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,DFSTROKE,DFRSKFCT,DFPRESNT,DFRPROG,...,DFBWLDYS,DFOCULO,DFEYELID,DFNEURAB,DFDOPRSP,DFRAPSPE,DFBULBAR,DFCTSCAN,DFMRI,DFATYP
0,358812401,V,3428,V04,DIAGFEAT,08/2012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0
1,364251801,V,3433,V04,DIAGFEAT,09/2012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,0,0,0.0
2,366180201,V,3166,V04,DIAGFEAT,10/2012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,N,0,0.0
3,369126801,V,3162,V04,DIAGFEAT,10/2012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,N,N,0.0
4,362799201,V,3432,ST,DIAGFEAT,08/2012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,0,0,0.0


### 'Clinical_Diagnosis_and_Management.csv': Feat-PD

In [100]:
### 'Clinical_Diagnosis_and_Management.csv': Feat-PD
CLINDX=dict_files[files_names[2]]
CLINDX.head()
#CLINDX.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PSLVL', 'DCNOMTR', 'DCRTREM', 'DCRIGID', 'DCBRADY', 'DFPGDIST',
       'DFOTH', 'PRIMDIAG', 'OTHNEURO', 'CLDXCHNG', 'DXFDTIMG',
       'DXFCLSGN', 'DXFRSPMD', 'DXFNATHX', 'DXFOTH', 'DXFOTHCM', 'MNGCHNG',
       'MNGTRTSX', 'MNGTRIAL', 'MNGTRTOT', 'MNGDXTST', 'MNGNOTRT',
       'NEURWKDX']
CLINDX_sel=CLINDX[SEL]
CLINDX_sel.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PSLVL,DCNOMTR,DCRTREM,DCRIGID,...,DXFNATHX,DXFOTH,DXFOTHCM,MNGCHNG,MNGTRTSX,MNGTRIAL,MNGTRTOT,MNGDXTST,MNGNOTRT,NEURWKDX
0,317677601,V,3408,BL,CLINDX,10/2011,3,0,0.0,1.0,...,1.0,0.0,NaN,0,0.0,0.0,0.0,0.0,1.0,NaN
1,317614301,V,3520,BL,CLINDX,10/2011,2,0,1.0,1.0,...,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,1.0,NaN
2,318556601,V,3426,BL,CLINDX,10/2011,3,0,1.0,0.0,...,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,1.0,Sciatica
3,318628801,V,3512,V02,CLINDX,10/2011,2,0,1.0,1.0,...,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,NaN
4,318729401,V,3425,BL,CLINDX,10/2011,2,0,0.0,1.0,...,0.0,0.0,NaN,0,0.0,0.0,0.0,1.0,0.0,spinal stenosis


### 'General_Medical_History.csv': useful: Feat-Gen

In [23]:
MHXGEN=dict_files[files_names[3]]
#MHXGEN.head()
#MHXGEN.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MHCAT', 'MHROW', 'MHHX', 'MHACTRES', 'MHDIAGYR', 
     'PT_CODE', 'PT_NAME', 'HLT_NAME', 'SOCABBR1']
# I need probably to select less features here
MHXGEN_sel=MHXGEN[SEL]
MHXGEN_sel.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MHCAT,MHROW,MHHX,MHACTRES,MHDIAGYR,PT_CODE,PT_NAME,HLT_NAME,SOCABBR1
0,226698901,V,3400,SC,MHXGEN,06/2010,1f. ENT,1,1.0,1.0,1987.0,10015090.0,Epistaxis,Nasal disorders NEC,Resp
1,226700701,V,3400,SC,MHXGEN,06/2010,1l. Gynecologic/Urologic,1,1.0,1.0,2005.0,10013935.0,Dysmenorrhoea,Menstruation and uterine bleeding NEC,Repro
2,226701901,V,3400,SC,MHXGEN,06/2010,1p. Neurologic (other than disease under study),1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,226702601,V,3400,SC,MHXGEN,06/2010,1r. Allergy / Immunologic (Note drug allergies),2,1.0,1.0,1981.0,10020751.0,Hypersensitivity,Allergic conditions NEC,Immun
4,226870201,V,3401,SC,MHXGEN,06/2010,1o. Hemato/lymphatic,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### General_Physical_Exam: useful: Feat-Gen

In [101]:
GENPHYEX=dict_files[files_names[5]]
#GENPHYEX.head()
#GENPHYEX.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PESEQ', 'PECAT', 'ABNORM']
GENPHYEX_sel=GENPHYEX[SEL]
GENPHYEX_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PESEQ,PECAT,ABNORM
0,226890001,3402,SC,GENPHYEX,06/2010,3,Eyes,0.0
1,226890301,3402,SC,GENPHYEX,06/2010,6,Cardiovascular (including peripheral vascular),0.0
2,226890601,3402,SC,GENPHYEX,06/2010,9,Neurological,0.0
3,232699801,3406,SC,GENPHYEX,07/2010,8,Musculoskeletal,0.0
4,232700101,3406,SC,GENPHYEX,07/2010,11,Other (Specify location and describe),0.0


### 'General_Neurological_Exam.csv': useful: Feat-Gen

In [25]:
PENEURO=dict_files[files_names[7]]
#PENEURO.head()
#PENEURO.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MSRARSP', 'MSRACM', 'MSLARSP', 'MSLACM', 'MSRLRSP', 'MSRLCM',
       'MSLLRSP', 'MSLLCM', 'COFNRRSP', 'COFNRCM', 'COFNLRSP', 'COFNLCM',
       'COHSRRSP', 'COHSRCM', 'COHSLRSP', 'COHSLCM', 'SENRARSP', 'SENRACM',
       'SENLARSP', 'SENLACM', 'SENRLRSP', 'SENRLCM', 'SENLLRSP', 'SENLLCM',
       'RFLRARSP', 'RFLRACM', 'RFLLARSP', 'RFLLACM', 'RFLRLRSP', 'RFLRLCM',
       'RFLLLRSP', 'RFLLLCM', 'PLRRRSP', 'PLRRCM', 'PLRLRSP', 'PLRLCM']
PENEURO_sel=PENEURO[SEL]
PENEURO_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,MSRARSP,MSRACM,MSLARSP,MSLACM,MSRLRSP,...,RFLLARSP,RFLLACM,RFLRLRSP,RFLRLCM,RFLLLRSP,RFLLLCM,PLRRRSP,PLRRCM,PLRLRSP,PLRLCM
0,232741801,3404,SC,PENEURO,06/2010,0,NaN,0,NaN,0,...,1.0,NaN,1.0,NaN,1.0,NaN,0.0,NaN,0.0,NaN
1,232698601,3406,SC,PENEURO,07/2010,0,NaN,0,NaN,0,...,2.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,0.0,NaN
2,226889601,3402,SC,PENEURO,06/2010,0,NaN,0,NaN,0,...,2.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,0.0,NaN
3,226914401,3403,SC,PENEURO,06/2010,0,NaN,0,NaN,1,...,3.0,NaN,3.0,NaN,3.0,NaN,0.0,NaN,0.0,NaN
4,226840201,3400,SC,PENEURO,06/2010,0,NaN,0,NaN,0,...,2.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,1.0,NaN


### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD

In [103]:
PECN=dict_files[files_names[13]]
#PECN.head()
#PECN.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CN1RSP','CN2RSP', 'CN346RSP','CN5RSP',
     'CN7RSP', 'CN8RSP', 'CN910RSP',
       'CN11RSP', 'CN12RSP']
PECN_sel=PENEURO2[SEL]
PECN_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,CN1RSP,CN2RSP,CN346RSP,CN5RSP,CN7RSP,CN8RSP,CN910RSP,CN11RSP,CN12RSP
0,232741701,3404,SC,PENEURO,06/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,232698501,3406,SC,PENEURO,07/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,226889501,3402,SC,PENEURO,06/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,226914301,3403,SC,PENEURO,06/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,226840101,3400,SC,PENEURO,06/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 'PD_Features.csv': useful: Feat-PD

In [105]:
PDFEAT=dict_files[files_names[14]]
#PDFEAT.head()
#PDFEAT.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'SXMO',
       'SXYEAR', 'PDDXDT', 'PDDXEST', 'DXTREMOR', 'DXRIGID', 'DXBRADY',
       'DXPOSINS', 'DXOTHSX', 'DXOTHCM', 'DOMSIDE']
PDFEAT_sel=PDFEAT[SEL]
PDFEAT_sel.head()
#list(set(PDFEAT_sel['DXOTHCM'])) - see if we remove the comment feature or not.

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,SXMO,SXYEAR,PDDXDT,PDDXEST,DXTREMOR,DXRIGID,DXBRADY,DXPOSINS,DXOTHSX,DXOTHCM,DOMSIDE
0,226923001,3403,SC,PDFEAT,06/2010,6.0,2009,03/2010,DAY,0,1,0,0,0,NaN,2
1,226845501,3400,SC,PDFEAT,06/2010,3.0,2007,10/2009,ACT,1,1,1,0,0,NaN,1
2,226895901,3402,SC,PDFEAT,06/2010,7.0,2008,03/2010,DAY,1,0,1,0,0,NaN,2
3,232711801,3406,SC,PDFEAT,07/2010,6.0,2008,01/2010,DAY,1,1,1,0,0,NaN,1
4,241672401,3407,SC,PDFEAT,08/2010,1.0,2010,02/2010,DAY,1,1,1,0,0,NaN,1


### 'Vital_Signs.csv': useful: Feat-Gen

In [106]:
VITAL=dict_files[files_names[15]]
#VITAL.head()
#VITAL.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'WGTKG', 'HTCM', 'TEMPC', 'SYSSUP', 'DIASUP', 'HRSUP',
       'SYSSTND', 'DIASTND', 'HRSTND']
VITAL_sel=VITAL[SEL]
VITAL_sel.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,WGTKG,HTCM,TEMPC,SYSSUP,DIASUP,HRSUP,SYSSTND,DIASTND,HRSTND
0,408779301,10362,SC,VITAL,08/2013,NaN,NaN,36.0,135.0,65.0,78.0,114.0,71.0,77.0
1,605754701,10405,SC,VITAL,09/2014,NaN,NaN,36.9,137.0,81.0,64.0,150.0,80.0,73.0
2,411032401,10496,SC,VITAL,06/2013,NaN,NaN,36.1,174.0,90.0,93.0,176.0,86.0,98.0
3,409587001,10606,SC,VITAL,08/2013,NaN,NaN,36.6,170.0,92.0,51.0,170.0,61.0,58.0
4,448228101,10662,SC,VITAL,07/2014,NaN,NaN,37.5,132.0,80.0,76.0,130.0,78.0,80.0


## 6-Motor-Assessments - all of them can be useful. 
#### But we might make the choice to not take all of them. 
#### For assessing the disease state (the labels), I think we should focus on the UPDRS1,3,&4 and take the other assessments as features

In [29]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv'

In [30]:
NUPDR1SP=dict_files[files_names[0]]
NUPDR1SP.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1SLPN,NP1SLPD,NP1PAIN,NP1URIN,NP1CNST,NP1LTHD,NP1FATG,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232593201,V,3400,SC,NUPDRS1P,07/2010,1.0,1.0,1.0,3.0,1.0,2.0,1.0,3.0,07/2010,2010-07-13 16:42:58.0,NaN,07/2010
1,230957601,V,3403,SC,NUPDRS1P,07/2010,1.0,0.0,1.0,1.0,0.0,3.0,0.0,2.0,07/2010,2010-07-06 13:08:13.0,NaN,07/2010
2,233164501,V,3400,BL,NUPDRS1P,07/2010,1.0,0.0,1.0,2.0,1.0,1.0,1.0,4.0,07/2010,2010-07-15 17:09:39.0,NaN,07/2010
3,234837801,V,3404,BL,NUPDRS1P,07/2010,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,07/2010,2010-07-23 11:35:18.0,NaN,07/2010
4,232529501,V,3401,BL,NUPDRS1P,07/2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-13 13:36:12.0,NaN,07/2010


### 'TAP-PD_OPDM_Use_Questionnaire.csv'

In [31]:
TAPUSE=dict_files[files_names[1]]
TAPUSE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,UNDRSDIR,CONFCORR,FITSCHED,NDREMIND,AFFPPMI,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,328345501,V,3432,V01,TAPUSE,01/2012,0,3,0,0,2,01/2012,2012-01-05 07:27:06.0,NaN,01/2012
1,329150101,V,3433,V01,TAPUSE,01/2012,0,3,0,0,2,01/2012,2012-01-12 08:45:59.0,NaN,01/2012
2,329429701,V,3435,V01,TAPUSE,01/2012,0,3,0,0,2,01/2012,2012-01-14 16:39:59.0,NaN,01/2012
3,332894601,V,3218,V01,TAPUSE,02/2012,0,3,0,0,2,02/2012,2012-02-08 11:51:38.0,NaN,02/2012
4,335013101,V,3436,V01,TAPUSE,02/2012,0,3,0,1,2,02/2012,2012-02-23 13:12:10.0,NaN,02/2012


### 'MDS_UPDRS_Part_I.csv'

In [32]:
NUPDRS1=dict_files[files_names[2]]
NUPDRS1.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231798601,V,3402,SC,NUPDRS1,07/2010,1.0,0.0,0.0,1.0,0.0,2.0,0.0,07/2010,2010-07-09 12:00:30.0,NaN,07/2010
1,233164301,V,3400,BL,NUPDRS1,07/2010,1.0,0.0,0.0,2.0,3.0,2.0,0.0,07/2010,2010-07-15 17:04:33.0,NaN,07/2010
2,230955201,V,3403,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-06 13:03:54.0,NaN,07/2010
3,234837501,V,3404,BL,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-23 11:30:12.0,NaN,07/2010
4,232712901,V,3406,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,1.0,1.0,0.0,07/2010,2010-07-14 08:54:29.0,NaN,07/2010


### 'PASE_-_Household_Activity.csv'

In [33]:
PASE=dict_files[files_names[3]]
PASE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,LTHSWRK,HVYHSWRK,HMREPR,LAWNWRK,OUTGARDN,CAREGVR,WRKVL,WRKVLHR,WRKVLACT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,456171801,V,10874,BL,PASE,2,1,1.0,2,2.0,1.0,1.0,NaN,NaN,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
1,531597501,V,10874,V04,PASE,2,2,1.0,2,2.0,1.0,1.0,NaN,NaN,08/2015,2015-08-17 08:54:53.0,NaN,2015-08-17 08:55:04.0
2,585712501,V,10874,V06,PASE,2,2,1.0,2,2.0,1.0,1.0,NaN,NaN,08/2016,2016-08-24 11:52:50.0,NaN,2016-08-24 11:53:02.0
3,648651901,V,10874,V08,PASE,2,2,1.0,2,2.0,2.0,1.0,NaN,NaN,08/2017,2017-08-18 12:07:35.0,NaN,2017-08-18 12:11:04.0
4,416212301,V,12224,BL,PASE,2,2,1.0,2,2.0,1.0,2.0,2.0,2.0,10/2013,2013-10-30 12:23:09.0,NaN,2013-10-30 12:23:19.0


### 'Gait_Data___Arm_swing.csv' - not sure if we should consider this dataset.

In [34]:
GAITDATA=dict_files[files_names[4]]
#GAITDATA.head()

### 'MDS_UPDRS_Part_IV.csv'

In [35]:
NUPDRS4=dict_files[files_names[5]]
NUPDRS4.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NP4WDYSK,NP4DYSKI,NP4OFF,NP4FLCTI,NP4FLCTX,NP4DYSTN,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,267411701,V,3403,ST,NUPDRS4,01/2011,0.0,0.0,0.0,0.0,0.0,0.0,01/2011,2011-01-08 09:34:03.0,NaN,01/2011
1,280733601,V,3251,U01,NUPDRS4,02/2011,0.0,0.0,0.0,0.0,0.0,0.0,03/2011,2011-03-16 12:19:26.0,NaN,03/2011
2,304301401,V,3800,ST,NUPDRS4,07/2011,0.0,0.0,0.0,0.0,0.0,0.0,07/2011,2011-07-11 05:07:01.0,NaN,07/2011
3,303916001,V,3400,V04,NUPDRS4,07/2011,0.0,0.0,0.0,0.0,0.0,0.0,07/2011,2011-07-07 08:06:50.0,NaN,07/2011
4,306573401,V,3403,V04,NUPDRS4,07/2011,0.0,0.0,0.0,0.0,0.0,0.0,07/2011,2011-07-27 08:20:55.0,NaN,07/2011


### 'Modified_Schwab_+_England_ADL.csv'

In [36]:
MODSEADL=dict_files[files_names[6]]
MODSEADL.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MSEADLG,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,230954401,V,3403,SC,MODSEADL,06/2010,90.0,07/2010,2010-07-06 13:02:56.0,NaN,07/2010
1,229763901,V,3402,SC,MODSEADL,06/2010,95.0,06/2010,2010-06-30 13:49:58.0,NaN,06/2010
2,230939601,V,3400,SC,MODSEADL,06/2010,90.0,07/2010,2010-07-06 12:36:23.0,NaN,07/2010
3,233166301,V,3400,BL,MODSEADL,07/2010,90.0,07/2010,2010-07-15 17:20:22.0,NaN,07/2010
4,235635801,V,3406,SC,MODSEADL,07/2010,90.0,07/2010,2010-07-28 12:02:16.0,NaN,07/2010


### 'MDS_UPDRS_Part_III.csv'

In [37]:
NUPDRS3=dict_files[files_names[7]]
NUPDRS3.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,CMEDTM,EXAMTM,NP3SPCH,NP3FACXP,...,DYSKPRES,DYSKIRAT,NHY,ANNUAL_TIME_BTW_DOSE_NUPDRS,ON_OFF_DOSE,PD_MED_USE,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,324141901,V,3506,V03,NUPDRS3,10/2011,NaN,NaN,1.0,1.0,...,0.0,NaN,1.0,NaN,NaN,0,12/2011,2011-12-05 19:23:15.0,NaN,12/2011
1,324180301,V,3419,V02,NUPDRS3,12/2011,NaN,NaN,0.0,0.0,...,0.0,NaN,1.0,NaN,NaN,0,12/2011,2011-12-06 07:25:10.0,NaN,12/2011
2,324389901,V,3816,BL,NUPDRS3,12/2011,NaN,NaN,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,0,12/2011,2011-12-07 02:10:18.0,NaN,12/2011
3,324396501,V,3762,BL,NUPDRS3,11/2011,NaN,NaN,0.0,1.0,...,1.0,1.0,2.0,NaN,NaN,0,12/2011,2011-12-07 02:34:41.0,NaN,12/2011
4,324399901,V,3765,BL,NUPDRS3,11/2011,NaN,NaN,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,0,12/2011,2011-12-07 02:47:42.0,NaN,12/2011


### 'TAP-PD_OPDM_Assessment.csv'

In [38]:
TAPOPDM=dict_files[files_names[8]]
TAPOPDM.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,OPDMCMPL,DEVICESN,DIGITOGR,PACKEYBD,PEGBRD,XFRYN,XFRDT,OPDMTM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,312776601,V,3432,BL,TAPOPDM,09/2011,1,HVDG9341016,1.0,1.0,1.0,1.0,09/2011,NaN,09/2011,2011-11-18 13:02:26.0,NaN,09/2011
1,314756601,V,3433,BL,TAPOPDM,09/2011,1,HVDG9341006,1.0,1.0,1.0,1.0,09/2011,NaN,09/2011,2011-11-18 13:00:24.0,NaN,09/2011
2,318524501,V,3435,BL,TAPOPDM,10/2011,1,HVDG9353016,1.0,1.0,1.0,1.0,10/2011,NaN,10/2011,2011-11-18 13:03:51.0,NaN,10/2011
3,322801201,V,3436,BL,TAPOPDM,11/2011,1,HVDG9343009,1.0,1.0,1.0,1.0,11/2011,NaN,12/2011,2011-12-01 10:03:16.0,NaN,12/2011
4,325919401,V,3220,BL,TAPOPDM,12/2011,1,HVDG9341003,1.0,1.0,1.0,0.0,NaN,NaN,12/2011,2011-12-10 13:19:37.0,NaN,12/2011


### 'TAP-PD_Kinetics_Device_Testing.csv' - Idem for this one- not sure it is essential to consider it.

In [39]:
KINETICS=dict_files[files_names[9]]
#KINETICS.head()

### 'PASE_-_Leisure_Time_Activity.csv'

In [40]:
PASE=dict_files[files_names[10]]
PASE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,QUESTNO,ACTVOFT,ACTVSPEC,HRDAYFRQ,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,456171201,V,10874,BL,PASE,1,3.0,"reading, computer, TV",3.0,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
1,456171301,V,10874,BL,PASE,2,3.0,NaN,2.0,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
2,456171401,V,10874,BL,PASE,3,3.0,gardening,3.0,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
3,456171501,V,10874,BL,PASE,4,0.0,NaN,NaN,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
4,456171601,V,10874,BL,PASE,5,0.0,NaN,NaN,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0


### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv'

In [41]:
NUPDRS2P=dict_files[files_names[11]]
NUPDRS2P.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP2SPCH,NP2SALV,NP2SWAL,...,NP2HOBB,NP2TURN,NP2TRMR,NP2RISE,NP2WALK,NP2FREZ,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232529901,V,3401,BL,NUPDRS2P,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-13 13:37:01.0,NaN,07/2010
1,232713901,V,3406,SC,NUPDRS2P,07/2010,1.0,0.0,2.0,0.0,...,2.0,0.0,3.0,0.0,0.0,0.0,07/2010,2010-07-14 08:56:34.0,NaN,07/2010
2,231800401,V,3402,SC,NUPDRS2P,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,07/2010,2010-07-09 12:04:22.0,NaN,07/2010
3,232595801,V,3400,SC,NUPDRS2P,07/2010,1.0,0.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,07/2010,2010-07-13 16:48:22.0,NaN,07/2010
4,230957201,V,3403,SC,NUPDRS2P,07/2010,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,07/2010,2010-07-06 13:06:55.0,NaN,07/2010


## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

In [42]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'University_of_Pennsylvania_Smell_ID_Test.csv'

In [43]:
UPSIT=dict_files[files_names[0]]
UPSIT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,UPSITBK1,UPSITBK2,UPSITBK3,UPSITBK4,NORMATIVE_SCORE,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231754301,V,3403,BL,UPSIT,07/2010,4.0,2.0,1.0,2.0,Anosmia,NaN,07/2010,2010-07-09 09:53:39.0,NaN,07/2010
1,232551901,V,3401,BL,UPSIT,07/2010,9.0,9.0,5.0,4.0,Hyposmia,NaN,07/2010,2010-07-13 14:23:31.0,NaN,07/2010
2,233828501,V,3400,BL,UPSIT,07/2010,6.0,8.0,7.0,9.0,Hyposmia,NaN,07/2010,2010-07-20 07:37:29.0,NaN,07/2010
3,234847101,V,3404,BL,UPSIT,07/2010,8.0,6.0,9.0,10.0,Hyposmia,NaN,07/2010,2010-07-23 12:15:13.0,NaN,07/2010
4,236093901,V,3405,BL,UPSIT,07/2010,9.0,10.0,10.0,10.0,Normosmia,NaN,07/2010,2010-07-30 10:34:31.0,NaN,07/2010


### 'Features_of_REM_Behavior_Disorder.csv'

In [44]:
UPSIT=dict_files[files_names[1]]
UPSIT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,REMONSDT,REMONEST,RBDDXDT,RBDDXEST,...,ONBENZ,ONMLATON,ONSSRI,ONNORSRI,ONTRIADP,ONBTABLK,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,608680701,V,60048,SC,REMBHVDS,06/2014,01/1987,2.0,07/2007,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-21 08:10:43.0,NaN,12/2016
1,608900501,V,60033,SC,REMBHVDS,12/2016,06/1996,3.0,05/2014,1,...,0.0,1.0,1.0,1.0,0.0,0.0,12/2016,2016-12-22 09:13:32.0,NaN,12/2016
2,609531601,V,60065,SC,REMBHVDS,03/2014,01/2005,3.0,01/2010,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-29 04:02:29.0,NaN,12/2016
3,609625701,V,60146,SC,REMBHVDS,12/2014,01/2007,3.0,04/2011,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-30 01:06:15.0,NaN,12/2016
4,609524901,V,60059,SC,REMBHVDS,02/2014,12/2002,3.0,02/2011,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-28 23:53:04.0,NaN,12/2016


### 'Epworth_Sleepiness_Scale.csv

In [45]:
EPWORTH=dict_files[files_names[2]]
EPWORTH.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234842801,V,3404,BL,EPWORTH,07/2010,1.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,07/2010,2010-07-23 11:54:04.0,NaN,07/2010
1,233819801,V,3400,BL,EPWORTH,07/2010,1.0,2.0,2.0,1.0,3.0,3.0,0.0,1.0,0.0,07/2010,2010-07-20 07:17:01.0,NaN,07/2010
2,232533701,V,3401,BL,EPWORTH,07/2010,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,07/2010,2010-07-13 13:44:34.0,NaN,07/2010
3,236092201,V,3405,BL,EPWORTH,07/2010,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,07/2010,2010-07-30 10:24:35.0,NaN,07/2010
4,231743201,V,3403,BL,EPWORTH,07/2010,1.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,07/2010,2010-07-09 09:39:50.0,NaN,07/2010


### 'Letter_-_Number_Sequencing__PD_.csv'

In [46]:
LNSPD=dict_files[files_names[3]]
LNSPD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,LNS1A,LNS1B,LNS1C,LNS2A,...,LNS7A,LNS7B,LNS7C,LNS_TOTRAW,AGE_ASSESS_LNS,DVS_LNS,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233165701,V,3400,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10.0,39.0,9.0,07/2010,2010-07-15 17:16:27.0,NaN,07/2010
1,232532701,V,3401,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10.0,56.0,11.0,07/2010,2010-07-13 13:43:06.0,NaN,07/2010
2,234840901,V,3404,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,15.0,55.0,16.0,07/2010,2010-07-23 11:51:30.0,NaN,07/2010
3,231742701,V,3403,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10.0,68.0,11.0,07/2010,2010-07-09 09:38:07.0,NaN,07/2010
4,236091801,V,3405,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,9.0,63.0,10.0,07/2010,2010-07-30 10:21:29.0,NaN,07/2010


### 'Olfactory_UPSIT.csv'

In [47]:
OLFACT=dict_files[files_names[4]]
OLFACT.head()

,SUBJECT_ID,PROTOCOL_ID,COMPLT_DATE,SCENT_10_RESPONSE,SCENT_09_RESPONSE,SCENT_08_RESPONSE,SCENT_07_RESPONSE,SCENT_06_RESPONSE,SCENT_05_RESPONSE,SCENT_04_RESPONSE,...,SCENT_38_CORRECT,SCENT_37_CORRECT,SCENT_36_CORRECT,SCENT_35_CORRECT,SCENT_34_CORRECT,SCENT_33_CORRECT,SCENT_32_CORRECT,SCENT_31_CORRECT,TOTAL_CORRECT,UPSIIT_PRCNTGE
0,90000,P-PPMI Cohort,04/2013,2,3,2,1,2,3,4,...,1,1,0,1,1,1,1,1,38,98
1,90001,P-PPMI Cohort,04/2013,2,3,2,1,2,3,4,...,1,0,1,1,0,1,0,1,30,36
2,90002,P-PPMI Cohort,03/2013,2,3,2,1,2,1,4,...,1,1,1,1,0,1,1,1,28,36
3,90003,P-PPMI Cohort,04/2013,3,1,1,4,2,4,3,...,0,0,0,0,0,0,0,0,8,0
4,90004,P-PPMI Cohort,04/2013,1,3,2,1,2,3,4,...,1,1,0,1,1,1,0,1,32,75


### 'REM_Sleep_Disorder_Questionnaire.csv'

In [48]:
REMSLEEP=dict_files[files_names[5]]
REMSLEEP.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,DRMVIVID,DRMAGRAC,DRMNOCTB,...,NARCLPSY,DEPRS,EPILEPSY,BRNINFM,CNSOTH,CNSOTHCM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232534301,V,3401,BL,REMSLEEP,07/2010,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-13 13:46:12.0,NaN,07/2010
1,233821601,V,3400,BL,REMSLEEP,07/2010,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,NaN,07/2010,2010-07-20 07:19:50.0,NaN,07/2010
2,234843201,V,3404,BL,REMSLEEP,07/2010,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-23 11:55:28.0,NaN,07/2010
3,231743401,V,3403,BL,REMSLEEP,07/2010,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-09 09:41:25.0,NaN,07/2010
4,236092401,V,3405,BL,REMSLEEP,07/2010,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-30 10:27:21.0,NaN,07/2010


### 'Cognitive_Assessments.csv'

In [49]:
COGTIME=dict_files[files_names[6]]
COGTIME.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,HVLTRTTM,HVLTDRTM,LNORNTTM,SFTTM,LNSTM,SDMTM,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,517792401,V,3914,V06,COGTIME,04/2015,11:18:00,11:45:00,NaN,NaN,NaN,NaN,Only noted times for HVLT-R during this visit....,04/2015,2015-04-29 03:01:42.0,NaN,04/2015
1,518015001,V,3871,V06,COGTIME,04/2015,NaN,NaN,NaN,NaN,NaN,NaN,Not completed- test introduced only very recen...,04/2015,2015-04-30 01:38:31.0,NaN,05/2015
2,518059301,V,41291,BL,COGTIME,04/2015,18:00:00,18:20:00,18:25:00,18:30:00,18:33:00,18:43:00,NaN,04/2015,2015-04-30 06:26:32.0,NaN,04/2015
3,520658401,V,4114,V06,COGTIME,05/2015,14:58:00,15:38:00,15:12:00,15:18:00,15:26:00,15:32:00,NaN,05/2015,2015-05-05 08:43:23.0,NaN,05/2015
4,520722501,V,3446,V08,COGTIME,05/2015,09:58:00,10:17:00,10:03:00,10:05:00,10:09:00,10:12:00,NaN,05/2015,2015-05-05 12:00:25.0,NaN,05/2015


### 'SCOPA-AUT.csv'

In [50]:
SCOPAAUT=dict_files[files_names[7]]
SCOPAAUT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,SCAU1,SCAU2,SCAU3,...,SCAU26B,SCAU26BT,SCAU26C,SCAU26CT,SCAU26D,SCAU26DT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233827401,V,3400,BL,SCOPAAUT,07/2010,1.0,0.0,1.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:13.0,NaN,07/2010
1,232551401,V,3401,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010
2,234844601,V,3404,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010
3,236093701,V,3405,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010
4,231753901,V,3403,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,1.0,Enalapril,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010


### Cognitive_Categorization.csv'

In [51]:
COGCATG=dict_files[files_names[8]]
COGCATG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,COGDECLN,FNCDTCOG,COGSTATE,COGDXCL,RVWNPSY,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,365358701,V,3574,ST,COGCATG,10/2012,1.0,0.0,0.0,1.0,1.0,0.0,10/2012,2012-10-04 09:02:01.0,NaN,10/2012
1,355330201,V,3179,BL,COGCATG,07/2012,3.0,0.0,0.0,1.0,1.0,1.0,07/2012,2012-07-23 17:22:03.0,NaN,07/2012
2,351869301,V,3178,BL,COGCATG,06/2012,1.0,1.0,0.0,2.0,2.0,1.0,06/2012,2012-06-25 18:20:31.0,NaN,06/2012
3,362798601,V,3432,ST,COGCATG,08/2012,1.0,0.0,0.0,1.0,1.0,0.0,09/2012,2012-09-18 13:44:55.0,NaN,09/2012
4,364318401,V,3407,V06,COGCATG,09/2012,1.0,0.0,0.0,1.0,1.0,0.0,09/2012,2012-09-27 12:15:21.0,NaN,09/2012


### 'Hopkins_Verbal_Learning_Test.csv'

In [52]:
HVLT=dict_files[files_names[9]]
HVLT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,HVLTRT1,HVLTRT2,HVLTRT3,HVLTRDLY,...,comm,AGE_ASSESS_HVLT,DVT_TOTAL_RECALL,DVT_DELAYED_RECALL,DVT_RETENTION,DVT_RECOG_DISC_INDEX,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231741801,V,3403,BL,HVLT,07/2010,6.0,7.0,8.0,9.0,...,NaN,68.0,44.0,52.0,63.0,47.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
1,232531001,V,3401,BL,HVLT,07/2010,6.0,10.0,11.0,12.0,...,NaN,56.0,49.0,61.0,62.0,51.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
2,233165101,V,3400,BL,HVLT,07/2010,8.0,10.0,10.0,11.0,...,NaN,39.0,49.0,54.0,62.0,58.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
3,234838901,V,3404,BL,HVLT,07/2010,8.0,10.0,11.0,12.0,...,NaN,55.0,54.0,61.0,62.0,44.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
4,236090801,V,3405,BL,HVLT,07/2010,6.0,9.0,12.0,10.0,...,NaN,63.0,51.0,53.0,44.0,52.0,07/2010,2012-06-03 09:05:08.0,NaN,07/2010


### 'Montreal_Cognitive_Assessment__MoCA_.csv'

In [53]:
MOCA=dict_files[files_names[10]]
MOCA.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MCAALTTM,MCACUBE,MCACLCKC,MCACLCKN,...,MCAMONTH,MCAYR,MCADAY,MCAPLACE,MCACITY,MCATOT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226919901,V,3403,SC,MOCA,06/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,25.0,06/2010,2010-06-21 11:25:04.0,NaN,07/2010
1,232509801,V,3401,SC,MOCA,07/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,28.0,07/2010,2010-07-13 12:53:27.0,NaN,07/2010
2,232586001,V,3405,SC,MOCA,07/2010,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,29.0,07/2010,2010-07-13 16:23:49.0,NaN,07/2010
3,234494501,V,3406,SC,MOCA,07/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,29.0,07/2010,2010-07-22 11:01:44.0,NaN,07/2010
4,226892001,V,3402,SC,MOCA,06/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,30.0,06/2010,2010-06-21 10:51:41.0,NaN,06/2010


### 'Semantic_Fluency.csv'

In [54]:
SFT=dict_files[files_names[11]]
SFT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,VLTANIM,VLTVEG,VLTFRUIT,AGE_ASSESS_SFTANIM,DVS_SFTANIM,DVT_SFTANIM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233165501,V,3400,BL,SFT,07/2010,22,24,17,39.0,11.0,42.0,07/2010,2010-07-15 17:15:37.0,NaN,07/2010
1,232532101,V,3401,BL,SFT,07/2010,12,18,18,56.0,5.0,25.0,07/2010,2010-07-13 13:42:06.0,NaN,07/2010
2,234840401,V,3404,BL,SFT,07/2010,16,14,11,55.0,8.0,41.0,07/2010,2010-07-23 11:49:01.0,NaN,07/2010
3,231742501,V,3403,BL,SFT,07/2010,19,9,10,68.0,10.0,48.0,07/2010,2010-07-09 09:37:11.0,NaN,07/2010
4,236091601,V,3405,BL,SFT,07/2010,25,20,16,63.0,13.0,57.0,07/2010,2010-07-30 10:20:44.0,NaN,07/2010


### 'Geriatric_Depression_Scale__Short_.csv'

In [55]:
GDSSHORT=dict_files[files_names[12]]
GDSSHORT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,GDSSATIS,GDSDROPD,GDSEMPTY,GDSBORED,...,GDSMEMRY,GDSALIVE,GDSWRTLS,GDSENRGY,GDSHOPLS,GDSBETER,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234843501,V,3404,BL,GDSSHORT,07/2010,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,07/2010,2010-07-23 11:56:16.0,NaN,07/2010
1,233822201,V,3400,BL,GDSSHORT,07/2010,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-20 07:21:07.0,NaN,07/2010
2,232534901,V,3401,BL,GDSSHORT,07/2010,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,07/2010,2010-07-13 13:48:13.0,NaN,07/2010
3,231744001,V,3403,BL,GDSSHORT,07/2010,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,07/2010,2010-07-09 09:43:18.0,NaN,07/2010
4,236092601,V,3405,BL,GDSSHORT,07/2010,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,07/2010,2010-07-30 10:28:27.0,NaN,07/2010


### 'Symbol_Digit_Modalities.csv'

In [56]:
SDM=dict_files[files_names[13]]
SDM.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SDMTOTAL,SDMTVRSN,AGE_ASSESS_SDM,DVSD_SDM,DVT_SDM,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231742901,V,3403,BL,SDM,07/2010,32.0,1.0,68.0,-1.000,40.000000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
1,232533101,V,3401,BL,SDM,07/2010,53.0,1.0,56.0,0.625,56.250000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
2,233165901,V,3400,BL,SDM,07/2010,47.0,1.0,39.0,-0.667,43.330002,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
3,234842101,V,3404,BL,SDM,07/2010,50.0,1.0,55.0,0.250,52.500000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
4,236092001,V,3405,BL,SDM,07/2010,53.0,1.0,63.0,0.625,56.250000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010


### 'State-Trait_Anxiety_Inventory.csv'

In [57]:
STAI=dict_files[files_names[14]]
STAI.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,STAIAD1,STAIAD2,STAIAD3,STAIAD4,...,STAIAD35,STAIAD36,STAIAD37,STAIAD38,STAIAD39,STAIAD40,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234843901,V,3404,BL,STAI,07/2010,3.0,3.0,1.0,1.0,...,1.0,3.0,2.0,1.0,3.0,2.0,07/2010,2010-07-23 11:58:39.0,NaN,07/2010
1,233826001,V,3400,BL,STAI,07/2010,3.0,2.0,1.0,1.0,...,2.0,3.0,4.0,2.0,3.0,4.0,07/2010,2010-07-20 07:28:30.0,NaN,07/2010
2,232536101,V,3401,BL,STAI,07/2010,4.0,4.0,1.0,1.0,...,1.0,4.0,2.0,1.0,4.0,1.0,07/2010,2010-07-13 13:52:33.0,NaN,07/2010
3,236092801,V,3405,BL,STAI,07/2010,4.0,4.0,1.0,1.0,...,2.0,4.0,2.0,2.0,4.0,1.0,07/2010,2010-07-30 10:30:33.0,NaN,07/2010
4,231746001,V,3403,BL,STAI,07/2010,2.0,2.0,2.0,2.0,...,2.0,3.0,1.0,1.0,3.0,2.0,07/2010,2010-07-09 09:47:13.0,NaN,07/2010


### 'Benton_Judgment_of_Line_Orientation.csv'

In [58]:
BENTONOD=dict_files[files_names[15]]
BENTONOD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,BJLOT1,BJLOT2,BJLOT3,BJLOT4,...,COMM,JLO_TOTRAW,JLO_TOTCALC,AGE_ASSESS_JLO,DVS_JLO_MSSA,DVS_JLO_MSSAE,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233165301,V,3400,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,13.0,26.0,39.0,13.0,11.70,07/2010,2010-07-15 17:15:13.0,NaN,07/2010
1,232531901,V,3401,BL,BENTONOD,07/2010,0.0,NaN,1.0,NaN,...,NaN,9.0,18.0,56.0,8.0,5.97,07/2010,2010-07-13 13:41:35.0,NaN,07/2010
2,234839901,V,3404,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,15.0,30.0,55.0,16.0,15.92,07/2010,2010-07-23 11:46:15.0,NaN,07/2010
3,236091401,V,3405,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,14.0,28.0,63.0,14.0,12.80,07/2010,2010-07-30 10:20:02.0,NaN,07/2010
4,231742101,V,3403,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,13.0,26.0,68.0,13.0,11.70,07/2010,2010-07-09 09:36:16.0,NaN,07/2010


### 'QUIP_Current_Short.csv'

In [59]:
QUIPCS=dict_files[files_names[16]]
QUIPCS.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTINBOTH,TMGAMBLE,CNTRLGMB,TMSEX,...,CNTRLEAT,TMTORACT,TMTMTACT,TMTRWD,TMDISMED,CNTRLDSM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232537101,V,3401,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-13 13:56:29.0,NaN,07/2010
1,233826501,V,3400,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-20 07:29:44.0,NaN,07/2010
2,234844401,V,3404,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-23 11:59:49.0,NaN,07/2010
3,231746401,V,3403,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-09 09:48:15.0,NaN,07/2010
4,236093001,V,3405,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-30 10:31:15.0,NaN,07/2010


## 4-Imaging - dataset not to be used for the time being.

In [60]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/4-Imaging/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### FBB_Analysis_Data.csv

In [61]:
FBBANAL=dict_files[files_names[0]]
#FBBANAL.head()

### AV-133_Image_Metadata.csv

In [62]:
AV133META=dict_files[files_names[1]]
#AV133META.head()

### AV-133_Imaging.csv

In [63]:
AVIMAG=dict_files[files_names[2]]
#AVIMAG.head()

### DaTscan_Imaging.csv

In [64]:
DATSCAN=dict_files[files_names[3]]
#DATSCAN.head()

### SPECT_Scan_Information_Source_Document.csv

In [65]:
SPECTSCANINFO=dict_files[files_names[4]]
#SPECTSCANINFO.head()

### AV-133_SBR_Results.csv

In [66]:
AV133SBRRSLT=dict_files[files_names[5]]
#AV133SBRRSLT.head()

### Magnetic_Resonance_Imaging.csv

In [67]:
MRI=dict_files[files_names[6]]
#MRI.head()

### DTI_Regions_of_Interest

In [68]:
DTIROI=dict_files[files_names[7]]
#DTIROI.head()

### DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv

In [69]:
FBBMETA=dict_files[files_names[8]]
#FBBMETA.head()

### 'FBB_Metadata.csv'

In [70]:
FBBMETA=dict_files[files_names[9]]
#FBBMETA.head()

### 'MRI_Imaging_Data_Transfer_Information_Source_Document.csv'

In [71]:
MRIINFO=dict_files[files_names[10]]
#MRIINFO.head()

### Family_History_PD - most likely useful but not to be used for now.
#### Can be useful for detecting early PD

In [72]:
FAMHXPD=dict_files[files_names[2]]
#FAMHXPD.head()

### 'Signature_Form.csv' - most likely useful: Meds - but not to be used for now.
#### Can be useful for the meds log

In [73]:
SIG=dict_files[files_names[10]]
#SIG.head()

### Use of PD Medication: useful: Meds - but not to be used for now.

In [74]:
PDMEDUSE=dict_files[files_names[4]]
#PDMEDUSE.head()

### 'Concomitant_Medications.csv': useful: Meds - But not to be used for now.

In [75]:
CMED=dict_files[files_names[6]]
#CMED.head()

### 'Current_Medical_Conditions_Log.csv': useful: Meds - but not to be used for now. 

In [76]:
### 'Current_Medical_Conditions_Log.csv': Meds
CURRCOND=dict_files[files_names[8]]
#CURRCOND.head()

### 'Adverse_Event_Log.csv': useful: Meds - but not to be used for now.
#### Measures the side effects of the meds taken for PD.

In [77]:
LOG=dict_files[files_names[9]]
#LOG.head()

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds - but not to be used for now.

In [78]:
SURGPD=dict_files[files_names[12]]
#SURGPD.head()

IndexError: list index out of range